In [4]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji
from sklearn.metrics import *

In [5]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]=="fake"):
            
            #if the predicited class is negative
            if(Y_pred[i]=="fake"):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]=="real"):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Predicted Negative','Predicted Positive'])
    
    Confusion_Matrix.rename(index={0: "Actual Negative", 1: "Actual Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)

In [32]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)

In [33]:
def preprocess_text(tweet, flag_stemm=True, flag_lemm=False, stop_words_list=None):
#     print(tweet)
    tweet=emoji.demojize(tweet)
    tweet=tweet.replace(":"," ")
#     print(tweet)
    tweet=p.clean(tweet)
    ## clean (convert to lowercase and remove punctuations and characters and then strip and remove url)
    
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
#     print(tweet)

    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

In [34]:
df_training=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
df_training.drop(columns="id",inplace=True)
df_training
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets
vectorizer = TfidfVectorizer(ngram_range=(1,4),min_df=5)

In [48]:
Train_ngrams=vectorizer.fit_transform(df_training.tweet)


In [49]:
# for i in tqdm(range(len(tweets_test))):
#     #Removing Urls, mentions, hashtags
#     res=p.clean(tweets_test[i])
#     res = re.sub(r'[^\w\s]', '',res)
#     tweets_test[i]=res


In [50]:
df_validation=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))
df_validation.drop(columns="id",inplace=True)
df_validation
df_validation['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_validation['tweet'])


In [51]:
Validation_ngrams=vectorizer.transform(df_validation.tweet)

In [52]:
Validation_ngrams

<2140x5299 sparse matrix of type '<class 'numpy.float64'>'
	with 37215 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)


In [54]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9054402274964453

In [55]:
f1_score(y_pred,df_validation["label"],average="macro")
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.905607476635514
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9053571428571429
-----------------
Recall: 0.9135135135135135
-----------------
F1 Score: 0.9094170403587443

For the Negative CLass, Label:0
-----------------
Precision: 0.9058823529411765
-----------------
Recall: 0.8970873786407767
-----------------
F1 Score: 0.9014634146341464

--------
Macro Average F1 0.9054402274964453
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 924                 106
Actual Positive                  96                1014


In [56]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid",solver=""))
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)
f1_score(y_pred,df_validation["label"],average="macro")

In [57]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9246530121846757

In [58]:
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.9247663551401869
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9205357142857142
-----------------
Recall: 0.9347234814143246
-----------------
F1 Score: 0.9275753486279802

For the Negative CLass, Label:0
-----------------
Precision: 0.9294117647058824
-----------------
Recall: 0.914175506268081
-----------------
F1 Score: 0.921730675741371

--------
Macro Average F1 0.9246530121846757
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 948                  89
Actual Positive                  72                1031


In [71]:
df_test=pd.DataFrame(pd.read_csv("Constraint_English_Test - Sheet1.csv"))
df_test
df_test['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_test['tweet'])


<ipython-input-32-fb9ae317fd00>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)


In [60]:
Test_ngrams=vectorizer.transform(df_validation.tweet)

In [65]:
y_pred=clf.predict(Validation_ngrams)
f1_score(df_validation["label"],y_pred,average="macro")

0.9246530121846757

In [62]:
y_pred_test=clf.predict(Test_ngrams)


In [63]:
y_pred_test=pd.DataFrame(y_pred_test)

In [78]:
ans=pd.DataFrame(df_test["id"])
ans["label"]=y_pred_test
ans.to_csv("answer.txt",index=False)

In [50]:
from sklearn.neural_network import MLPClassifier
clf2 = MLPClassifier(random_state=1, max_iter=300,verbose=True,activation="logistic",solver="lbfgs")
clf2.fit(Train_ngrams,df_training["label"])
y_pred=clf2.predict(Validation_ngrams)

In [51]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9189328442414445

In [52]:
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.9191588785046729
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9285714285714286
-----------------
Recall: 0.9179170344218888
-----------------
F1 Score: 0.9232134931202841

For the Negative CLass, Label:0
-----------------
Precision: 0.9088235294117647
-----------------
Recall: 0.9205561072492552
-----------------
F1 Score: 0.9146521953626049

--------
Macro Average F1 0.9189328442414445
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 927                  80
Actual Positive                  93                1040


In [81]:

from pysbd.utils import PySBDFactory

import urllib
import requests, re, spacy
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np
import csv
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords
from googletrans import Translator

translator = Translator()
ua = UserAgent()
nlp = spacy.load("en_core_web_sm")
# f = open('CommonWords.txt')
# commonEngWords = f.read().splitlines()
# f.close()
popular_links = [
        "nytimes", "wsj", "huffpost", "washingtonpost","time","republicworld",
        "latimes", "reuters", "abcnews", "usatoday",
        "bloomberg", "nbcnews", "dailymail", "theguardian",
        "thesun", "mirror", "telegraph", "bbc",
        "thestar", "theglobeandmail", "forbes",
        "cnbc", "chinadaily", "nypost", "usnews",
        "timesofindia", "thehindu", "hindustantimes",
        "cbsnews", "sfgate", "thehill", "thedailybeast",
        "newsweek", "theatlantic", "nzherald", "vanguardngr",
        "dailysun", "thejakartapost", "thestar",
        "straitstimes", "bangkokpost", "japantimes",
        "thedailystar", "scmp", "yahoo.com/news", "news.google"
        ]


def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])





def get_search_result(query, number_result = 20):
    query = urllib.parse.quote_plus(query) # Format into URL encoding
    
    
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")
    
    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
    
    links = []
    titles = []
    descriptions = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
            
            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)
        # Next loop if one element is not present
        except:
            continue
    return links, titles, descriptions

 
def clean_links(links, titles, descriptions):
    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa',l)
    
        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        clean_links.append(clean.group(1))
    
    # Remove the corresponding titles & descriptions
    for x in to_remove:
        del titles[x]
        del descriptions[x]
    return clean_links, titles, descriptions


def filter_links(links, titles, descriptions):
    to_remove = []
    for i, l in enumerate(links):
        if not any(a in l for a in popular_links):
            to_remove.append(i)
            continue
    
    # Remove the corresponding titles & descriptions
    links = [l for i,l in enumerate(links) if i not in to_remove]
    titles = [t for i,t in enumerate(titles) if i not in to_remove]
    links = [l for i,d in enumerate(descriptions) if i not in to_remove]
    return links, titles, descriptions
    

def valid_links(links):
    no_link = 0
    for link in links:
        if any(implink in link for implink in popular_links):
            no_link = no_link+1
    return no_link/len(links)


def split_sentence(text):
    nlp = spacy.blank('en')
    nlp.add_pipe(PySBDFactory(nlp))
    doc = nlp(text)
    return([sent.text for sent in doc.sents if sent.text.isspace()==False])


def valid_description(query, descriptions):
    temp_query = preprocessing(query)
    sentences = split_sentence(temp_query)
    score = []
    for q in query:
        temp = [levenshtein_ratio_and_distance(q, sent,ratio_calc = True) for sent in sentences]
        score.append(max(temp))
    return sum(score)/len(score)


def preprocessing(text):
    p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY)
    text = p.clean(text)
    text = remove_stopwords(text)
    text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('@','').replace('#','. ').replace('&amp;', 'and')
    return text


def translate_text(text):
    result = translator.translate('Mitä sinä teet')
    return result.text

    


In [114]:
score=[]
for query in tqdm(df_validation.tweet):
    links, titles, descriptions = get_search_result(query)
    links, titles, descriptions = filter_links(links, titles, descriptions)
    #descriptions = [preprocessing(desc) for desc in descriptions]
    if len(titles)>0:
        titles = [preprocessing(title.lower()) for title in titles]
        desc_score = valid_description(query, titles)
        score.append(desc_score)
    else:
        score.append(0)

100%|██████████| 2140/2140 [27:44<00:00,  1.29it/s]


In [111]:
lis=score2+score+score3
# len(score3)+len(score2)+len(score)
# len(lis)
# len(score3)

In [115]:
import pickle
file=open("score_validaiton",'wb')
pickle.dump(score,file)
file.close()

In [116]:
file=open("score_validaiton",'rb')
s=pickle.load(file)
file.close()
len(s)

2140

In [137]:
train_lis=[]
for i in lis:
        train_lis.append([i])
valid_lis=[]
for i in score:
        valid_lis.append([i])


In [138]:
from scipy.sparse import csr_matrix, hstack

csr_TrainFeature_without_ngram=csr_matrix(train_lis)
csr_ValidFeature_without_ngram=csr_matrix(valid_lis)
TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])
ValidFeatureFinal=hstack([Validation_ngrams,csr_ValidFeature_without_ngram])

In [152]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)

In [153]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9246530121846757

In [154]:
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.9247663551401869
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9205357142857142
-----------------
Recall: 0.9347234814143246
-----------------
F1 Score: 0.9275753486279802

For the Negative CLass, Label:0
-----------------
Precision: 0.9294117647058824
-----------------
Recall: 0.914175506268081
-----------------
F1 Score: 0.921730675741371

--------
Macro Average F1 0.9246530121846757
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 948                  89
Actual Positive                  72                1031
